# Week 7 Lab: The U-Shop Studio (Real-World Workflow)
## Single Image → Ecommerce Listing with Brand Guardrails

**U-Shop Context:** The University of Utah campus store is migrating to ecommerce. The store wants to automate product creation: given **one product image**, the AI should create a **standardized listing** with consistent brand voice, format, and constraints. This notebook follows a realistic workflow used in production.

**Real Workflow Order:**
1. Define brand policy and output format
2. Create the data contract (Pydantic)
3. Build the agent with the system prompt
4. Run on a real image
5. Add Chain-of-Thought to reduce logo mistakes
6. Add few-shot examples to stabilize tone
7. Build a golden test suite and iterate
8. Final export fields for ecommerce


## Step 0: Install Required Tools
We use **pydantic-ai** for the agent and **pydantic** for the contract.


In [ ]:
!pip install pydantic-ai -q
print("✅ Installed pydantic-ai")


## Step 0.5: Set Your API Key

You must set your OpenAI API key before running the agent.

- **Option A (Notebook):** set the key in a cell (temporary)
- **Option B (Terminal):** export the key once for your session


In [1]:
# Option A: set API key in the notebook (temporary for this session)
import os
import getpass

# Replace with your real key or use the secure prompt below
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"

# Secure option (recommended): prompt without showing the key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

print("✅ API key set for this notebook session")

# Option B (Terminal, outside notebook):
# export OPENAI_API_KEY="YOUR_API_KEY_HERE"


✅ API key set for this notebook session


## Step 1: Define Brand Policy + Output Format
**Concept:** The system prompt is the constitution. It locks non‑negotiable rules.

**U-Shop rules:**
- Never set prices below **$5.00**
- Always use the full name **"University of Utah"**
- Tone must be spirited and professional
- Output must follow the contract fields


In [2]:
system_prompt_base = (
    "You are the University of Utah Brand Manager. "
    "Rules: never set prices below $5.00. "
    "Always use 'University of Utah' in descriptions. "
    "Tone: spirited, professional, and concise."
)

print(system_prompt_base)


You are the University of Utah Brand Manager. Rules: never set prices below $5.00. Always use 'University of Utah' in descriptions. Tone: spirited, professional, and concise.


## Step 2: Create the Data Contract (Pydantic)
**Concept:** The contract defines the exact output fields and types.


In [3]:
from pydantic import BaseModel, Field

class UShopProduct(BaseModel):
    product_name: str = Field(description="Professional product name")
    has_interlocking_u: bool = Field(description="True if logo is present")
    material: str = Field(description="Primary material")
    marketing_copy: str = Field(description="2 sentences, spirited tone, ends with #GoUtes")
    price: float = Field(ge=5.0, description="Price in USD, must be at least $5")

print("✅ Contract ready:", UShopProduct.__name__)


✅ Contract ready: UShopProduct


## Step 3: Build the Agent (Prompt + Contract)
**Concept:** An agent = system prompt + contract.


In [5]:
from pydantic_ai import Agent

agent = Agent(
    "openai:gpt-4o-mini",
    output_type=UShopProduct,
    system_prompt=system_prompt_base
)

print("🤖 Agent ready")


🤖 Agent ready


## Step 4: Run on a Real Image
This is the first **real** listing generation.


In [7]:
from pydantic_ai import BinaryContent

image_path = "hoodie_clear.jpg"  # update if needed

with open(image_path, "rb") as f:
    image_bytes = f.read()

result = await agent.run([
    "Create a full U-Shop product listing for this image.",
    BinaryContent(data=image_bytes, media_type="image/jpeg")
])

from pprint import pprint

# Pretty print the structured output
try:
    pprint(result.data)
except AttributeError:
    pprint(result)


RuntimeError: This event loop is already running

## Step 5: Add Chain-of-Thought for Logo Accuracy
**Concept:** A reasoning checklist reduces false logo detection.


In [ ]:
cot_appendix = (
    "Logo check steps: "
    "1) Locate chest area. "
    "2) Identify white pixels. "
    "3) Compare shape to a 'U'. "
    "4) Decide yes/no."
)

system_prompt_cot = system_prompt_base + " " + cot_appendix

agent_cot = Agent(
    "openai:gpt-4o-mini",
    result_type=UShopProduct,
    system_prompt=system_prompt_cot
)

print("✅ CoT agent ready")


## Step 6: Add Few-Shot Examples for Tone
**Concept:** Examples stabilize writing style.


In [ ]:
few_shot_examples = (
    "Example 1: Show your Ute pride with this premium Crimson hoodie built for game day and everyday. #GoUtes
"
    "Example 2: Celebrate Utah tradition in a soft, campus-ready tee that brings spirit to every class. #GoUtes
"
    "Example 3: Clean lines, bold color, and classic Utes energy in one timeless sweatshirt. #GoUtes
"
)

system_prompt_full = system_prompt_cot + " " + "Use this tone: " + few_shot_examples

agent_full = Agent(
    "openai:gpt-4o-mini",
    result_type=UShopProduct,
    system_prompt=system_prompt_full
)

print("✅ CoT + Few-Shot agent ready")


## Step 7: Golden Test Suite (Regression)
**Concept:** A prompt is good only if it passes all tests.


In [ ]:
from pydantic_ai import BinaryContent

image_path = "hoodie_clear.jpg"  # update if needed

with open(image_path, "rb") as f:
    image_bytes = f.read()

result = await agent.run([
    "Create a full U-Shop product listing for this image.",
    BinaryContent(data=image_bytes, media_type="image/jpeg")
])

from pprint import pprint

# Pretty print the structured output
try:
    pprint(result.data)
except AttributeError:
    pprint(result)


## Step 8: Final Export Fields for Ecommerce
This is the clean set of fields your ecommerce system needs.


In [ ]:
# Example of extracting fields for a product listing
final_listing = {
    "title": result.product_name,
    "description": result.marketing_copy,
    "price": result.price,
    "material": result.material,
    "has_interlocking_u": result.has_interlocking_u,
}

print(final_listing)
